In [10]:
import os
import cv2
import numpy as np
import torch
from PIL import Image
import groundingdino.datasets.transforms as T
from groundingdino.util.inference import load_model, load_image, predict, annotate

In [2]:
import groundingdino.datasets.transforms as T

In [6]:
HOME = os.getcwd()
# set model configuration file path
# CONFIG_PATH = os.path.join(HOME, "groundingdino/config/GroundingDINO_SwinT_OGC.py")

# set model weight file ath
WEIGHTS_PATH = 'GroundingDINO/weights/groundingdino_swint_ogc.pth'

# set text prompt
TEXT_PROMPT = "glass with lid"

# set box and text threshold values
BOX_TRESHOLD = 0.35
TEXT_TRESHOLD = 0.25

In [25]:
model = load_model("GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py", "GroundingDINO/weights/groundingdino_swint_ogc.pth")

/Users/willrathgeb/CalHacks2023/calHacks2023/venv/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3527.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


final text_encoder_type: bert-base-uncased


In [26]:
while True:
     # ESP32 URL
    URL = 'http://172.20.10.6'
    AWB = True

    # Face recognition and opencv setup
    cap = cv2.VideoCapture(URL + ":81/stream")
    
    ret, frame = cap.read()
    # create a transform function by applying 3 image transaformations
    transform = T.Compose(
        [
            T.RandomResize([800], max_size=1333),
            T.ToTensor(),
            T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]
    )
    # convert frame to a PIL object in RGB space
    image_source = Image.fromarray(frame).convert("RGB")
    # convert the PIL image object to a transform object
    image_transformed, _ = transform(image_source, None)
    
    image_source.save('test.jpg')

    # predict boxes, logits, phrases
    boxes, logits, phrases = predict(
    model=model, 
    image=image_transformed, 
    caption=TEXT_PROMPT, 
    box_threshold=BOX_TRESHOLD, 
    text_threshold=TEXT_TRESHOLD,
    device='cpu')
    
    # annotate the image
    annotated_frame = annotate(image_source=frame, boxes=boxes, logits=logits, phrases=phrases)
    # display the output
    out_frame = cv2.cvtColor(annotated_frame, cv2.COLOR_BGR2RGB)
    cv2.imshow('frame', out_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


/Users/willrathgeb/CalHacks2023/calHacks2023/venv/lib/python3.9/site-packages/transformers/modeling_utils.py:905: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/Users/willrathgeb/CalHacks2023/calHacks2023/venv/lib/python3.9/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/Users/willrathgeb/CalHacks2023/calHacks2023/venv/lib/python3.9/site-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


KeyboardInterrupt: 

: 

In [13]:
from groundingdino.util.inference import load_model, load_image, predict, annotate

In [16]:
# Official GitHub implementation
from groundingdino.util.inference import load_model, load_image, predict, annotate
import cv2

IMAGE_PATH = "weights/dog-3.jpeg"
TEXT_PROMPT = "chair . person . dog ."
BOX_TRESHOLD = 0.35
TEXT_TRESHOLD = 0.25

image_source, image = load_image(IMAGE_PATH)

boxes, logits, phrases = predict(
    model=model,
    image=image,
    caption=TEXT_PROMPT,
    box_threshold=BOX_TRESHOLD,
    text_threshold=TEXT_TRESHOLD
)

annotated_frame = annotate(image_source=image_source, boxes=boxes, logits=logits, phrases=phrases)
cv2.imwrite("annotated_image.jpg", annotated_frame)


FileNotFoundError: [Errno 2] No such file or directory: 'weights/dog-3.jpeg'